A Name Generator using simple rnn network

In [44]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/names/names/README
/kaggle/input/names/names/female.txt
/kaggle/input/names/names/male.txt


lets read names of male

In [45]:
df_male=pd.read_csv('../input/names/names/male.txt')

As i have not sepcified the column name it assumed aamir to be column name. Thats fine we will also assume the same


In [46]:
df_male['Aamir']=df_male['Aamir'].str.lower()

lets create a text array which actually stores all are names 

In [47]:
text=df_male['Aamir'].to_numpy()

As Its clear that all names are not of equal length , so lets make them of equal length by giving a padding of ' '.we would require length of longest string

In [48]:
maxlen = len(max(text, key=len))

In [49]:
for i in range(len(text)):
  while len(text[i])<maxlen:
      text[i] += ' '

okay now lets create a dictionary. set helps us to only keep unique letters. 

In [50]:
chars=set(''.join(text))

As Rnn can only deal with numeric data we will create a dictionary for characters to int and vice versa. Then we will create one hot vector for each character in each name

In [51]:
int2char = dict(enumerate(chars))

In [52]:
int2char

{0: 'h',
 1: 'y',
 2: 'v',
 3: 'r',
 4: 'e',
 5: 'u',
 6: 'm',
 7: 'j',
 8: 'l',
 9: 's',
 10: 'b',
 11: 'd',
 12: 'p',
 13: 'z',
 14: 'w',
 15: 'c',
 16: 'o',
 17: 'q',
 18: 'x',
 19: 't',
 20: 'k',
 21: '-',
 22: 'a',
 23: 'i',
 24: ' ',
 25: 'g',
 26: 'f',
 27: 'n'}

In [53]:
import torch
from torch import nn

In [54]:
device=torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [55]:
char2int = {char: ind for ind, char in int2char.items()}

In our model for name generation, we will be providing a intial letter (lets say 'p') and Model shall generate rest of word(like 'patrick').
so input will be all letters of words except the last one.We dont need to input last letter as it will not have any prediction.similary we dont need first letter as output too.

In [56]:
# Creating lists that will hold our input and target sequences
input_seq = []
target_seq = []

for i in range(len(text)):
    # Remove last character for input sequence
  input_seq.append(text[i][:-1])
    
    # Remove first character for target sequence
  target_seq.append(text[i][1:])
  print("Input Sequence: {}\nTarget Sequence: {}".format(input_seq[i], target_seq[i]))

Input Sequence: aaron         
Target Sequence: aron          
Input Sequence: abbey         
Target Sequence: bbey          
Input Sequence: abbie         
Target Sequence: bbie          
Input Sequence: abbot         
Target Sequence: bbot          
Input Sequence: abbott        
Target Sequence: bbott         
Input Sequence: abby          
Target Sequence: bby           
Input Sequence: abdel         
Target Sequence: bdel          
Input Sequence: abdul         
Target Sequence: bdul          
Input Sequence: abdulkarim    
Target Sequence: bdulkarim     
Input Sequence: abdullah      
Target Sequence: bdullah       
Input Sequence: abe           
Target Sequence: be            
Input Sequence: abel          
Target Sequence: bel           
Input Sequence: abelard       
Target Sequence: belard        
Input Sequence: abner         
Target Sequence: bner          
Input Sequence: abraham       
Target Sequence: braham        
Input Sequence: abram         
Target Sequence: bram   

Target Sequence: onal          
Input Sequence: donald        
Target Sequence: onald         
Input Sequence: donn          
Target Sequence: onn           
Input Sequence: donnie        
Target Sequence: onnie         
Input Sequence: donny         
Target Sequence: onny          
Input Sequence: donovan       
Target Sequence: onovan        
Input Sequence: dorian        
Target Sequence: orian         
Input Sequence: dory          
Target Sequence: ory           
Input Sequence: doug          
Target Sequence: oug           
Input Sequence: douggie       
Target Sequence: ouggie        
Input Sequence: dougie        
Target Sequence: ougie         
Input Sequence: douglas       
Target Sequence: ouglas        
Input Sequence: douglass      
Target Sequence: ouglass       
Input Sequence: douglis       
Target Sequence: ouglis        
Input Sequence: dov           
Target Sequence: ov            
Input Sequence: doyle         
Target Sequence: oyle          
Input Sequence: drake  

Target Sequence: odge          
Input Sequence: rodger        
Target Sequence: odger         
Input Sequence: rodney        
Target Sequence: odney         
Input Sequence: rodolfo       
Target Sequence: odolfo        
Input Sequence: rodolph       
Target Sequence: odolph        
Input Sequence: rodolphe      
Target Sequence: odolphe       
Input Sequence: rodrick       
Target Sequence: odrick        
Input Sequence: rodrigo       
Target Sequence: odrigo        
Input Sequence: rodrique      
Target Sequence: odrique       
Input Sequence: rog           
Target Sequence: og            
Input Sequence: roger         
Target Sequence: oger          
Input Sequence: rogers        
Target Sequence: ogers         
Input Sequence: roice         
Target Sequence: oice          
Input Sequence: roland        
Target Sequence: oland         
Input Sequence: rolando       
Target Sequence: olando        
Input Sequence: rolf          
Target Sequence: olf           
Input Sequence: rolfe  

Input Sequence: stig          
Target Sequence: tig           
Input Sequence: stillman      
Target Sequence: tillman       
Input Sequence: stillmann     
Target Sequence: tillmann      
Input Sequence: sting         
Target Sequence: ting          
Input Sequence: stinky        
Target Sequence: tinky         
Input Sequence: stirling      
Target Sequence: tirling       
Input Sequence: stu           
Target Sequence: tu            
Input Sequence: stuart        
Target Sequence: tuart         
Input Sequence: sturgis       
Target Sequence: turgis        
Input Sequence: sullivan      
Target Sequence: ullivan       
Input Sequence: sully         
Target Sequence: ully          
Input Sequence: sumner        
Target Sequence: umner         
Input Sequence: sunny         
Target Sequence: unny          
Input Sequence: sutherland    
Target Sequence: utherland     
Input Sequence: sutton        
Target Sequence: utton         
Input Sequence: sven          
Target Sequence: ven    

In [57]:
for i in range(len(text)):
    input_seq[i] = [char2int[character] for character in input_seq[i]]
    target_seq[i] = [char2int[character] for character in target_seq[i]]


One Hotcoding the words

In [58]:
dict_size = len(char2int)
seq_len = maxlen - 1
batch_size = len(text)

def one_hot_encode(sequence, dict_size, seq_len, batch_size):
    # Creating a multi-dimensional array of zeros with the desired output shape
    features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)
    
    # Replacing the 0 at the relevant character index with a 1 to represent that character
    for i in range(batch_size):
        for u in range(seq_len):
            features[i, u, sequence[i][u]] = 1
    return features

In [59]:
input_seq = one_hot_encode(input_seq, dict_size, seq_len, batch_size)

Our input seqeunce looks like this

In [60]:
input_seq[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0

Import neccerary modules from pytorch

In [61]:
input_seq = torch.from_numpy(input_seq).to(device)
target_seq = torch.Tensor(target_seq).to(device)

Create The model

it is a simple nn with a rnn and a fully connected layer

In [62]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        hidden = self.init_hidden(batch_size)

        out, hidden = self.rnn(x, hidden.to(device))
        
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
        hidden.to(device)
        return hidden

Create instance of model 

In [63]:
model = Model(input_size=dict_size, output_size=dict_size, hidden_dim=12, n_layers=1)

model.to(device)
# Define hyperparameters
n_epochs = 25000
lr=0.01

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Train the model. Longer you train better the results will be

In [64]:
# Training Run
for epoch in range(1, n_epochs):
    optimizer.zero_grad() 
    input_seq.to(device)
    output, hidden = model(input_seq)
    loss = criterion(output, target_seq.view(-1).long())
    loss.backward() 
    optimizer.step() 
    
    if epoch%1000 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

Epoch: 1000/30000............. Loss: 0.8772
Epoch: 2000/30000............. Loss: 0.8657
Epoch: 3000/30000............. Loss: 0.8607
Epoch: 4000/30000............. Loss: 0.8571
Epoch: 5000/30000............. Loss: 0.8551
Epoch: 6000/30000............. Loss: 0.8540
Epoch: 7000/30000............. Loss: 0.8529
Epoch: 8000/30000............. Loss: 0.8520
Epoch: 9000/30000............. Loss: 0.8515
Epoch: 10000/30000............. Loss: 0.8509
Epoch: 11000/30000............. Loss: 0.8501
Epoch: 12000/30000............. Loss: 0.8488
Epoch: 13000/30000............. Loss: 0.8481
Epoch: 14000/30000............. Loss: 0.8477
Epoch: 15000/30000............. Loss: 0.8479
Epoch: 16000/30000............. Loss: 0.8473
Epoch: 17000/30000............. Loss: 0.8472
Epoch: 18000/30000............. Loss: 0.8469
Epoch: 19000/30000............. Loss: 0.8470
Epoch: 20000/30000............. Loss: 0.8470
Epoch: 21000/30000............. Loss: 0.8467
Epoch: 22000/30000............. Loss: 0.8463
Epoch: 23000/30000.

Predict the output. Note as we have used max it will only generate a particular word for a particular start word

In [71]:
def predict(model, character):
    # One-hot encoding our input to fit into the model
    character = np.array([[char2int[c] for c in character]])
    character = one_hot_encode(character, dict_size, character.shape[1], 1)
    character = torch.from_numpy(character).to(device)
    
    out, hidden = model(character)

    prob = nn.functional.softmax(out[-1], dim=0).data
    # Taking the class with the highest probability score from the output
    char_ind = torch.max(prob, dim=0)[1].item()

    return int2char[char_ind], hidden


In [72]:
def sample(model, out_len, start='hey'):
    model.eval() # eval mode
    start = start.lower()
    # First off, run through the starting characters
    chars = [ch for ch in start]
    size = out_len - len(chars)
    # Now pass in the previous characters and get a new one
    for ii in range(size):
        char, h = predict(model, chars)
        chars.append(char)

    return ''.join(chars)

In [76]:
sample(model, 15, 'p')

'parth          '

In [68]:
a=torch.tensor([1,2,3,4,5])

In [69]:
import random

In [70]:
torch.topk(a,5)[1][random.randint(0,5)]

IndexError: index 5 is out of bounds for dimension 0 with size 5

In [79]:
torch.save(model.state_dict(), '/kaggle/working/params.pt')

FileNotFoundError: [Errno 2] No such file or directory: '../output/params.pt'